In [1]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D

num_classes = 120
row , col = 64,64
batch_size = 32

Using TensorFlow backend.


In [2]:
train_dir = './fruits-360_dataset/fruits-360/Training'
test_dir = './fruits-360_dataset/fruits-360/Test'

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(row,col),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(row,col),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = False
)

Found 60498 images belonging to 120 classes.
Found 20622 images belonging to 120 classes.


In [3]:
model = Sequential()

model.add(Conv2D(32,(3,3),padding = 'same',input_shape = (row,col,3)))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),padding = 'same'))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# model.add(Conv2D(128,(3,3),padding = 'same'))
# model.add(Activation('relu'))
# model.add(Conv2D(128,(3,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

print(model.summary())






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 31, 32)        0         
______________________

In [4]:
from keras.optimizers import RMSprop,SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('/Users/sriuj/Documents/Projects/openCV/Start/Fruit_Classifier/100sizemodel.h5',
                            monitor = 'val_loss',
                            mode = 'min',
                            save_best_only=True,
                            verbose=1)
earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 3,
                          verbose = 1,
                         restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor = 0.2,
                             min_delta = 0.0001,
                             patience = 3,
                             verbose = 1)

callbacks = [earlystop,checkpoint]

model.compile(loss = 'categorical_crossentropy',
             optimizer = RMSprop(lr=0.001),
             metrics = ['accuracy'])

train_samples = 60498
test_samples = 20622
epochs = 10
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=test_generator,
    validation_steps=test_samples // batch_size)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





1890/1890 [==============================] - 517s 273ms/step - loss: 1.8045 - acc: 0.4844 - val_loss: 0.4940 - val_acc: 0.8502

Epoch 00001: val_loss improved from inf to 0.49404, saving model to /Users/sriuj/Documents/Projects/openCV/Start/Fruit_Classifier/100sizemodel.h5
Epoch 2/10
1890/1890 [==============================] - 512s 271ms/step - loss: 0.5438 - acc: 0.8331 - val_loss: 0.1692 - val_acc: 0.9485

Epoch 00002: val_loss improved from 0.49404 to 0.16925, saving model to /Users/sriuj/Documents/Projects/openCV/Start/Fruit_Classifier/100sizemodel.h5
Epoch 3/10
1890/1890 [==============================] - 515s 272ms/step - loss: 0.3892 - acc: 0.8857 - val_loss: 0.1898 - val_acc: 0.9577

Epoch 00003: val_loss did not improve from 0.16925
Epoch 4/10
1890/1890 [==============================] - 510s 270ms/step - loss: 0.3860 - acc: 0.8954 - val_loss: 0.4127 - val_acc: 0